In [1]:
import pandas as pd
import numpy as np
from scipy.integrate import trapz
from scipy.signal import find_peaks
from lets_plot import *
import pingouin as pg

LetsPlot.setup_html()

# sample information

In [5]:
# importing dataframe
df = pd.read_csv(r"C:\Users\bvenn\OneDrive\Desktop\Publikation\Repository\new_repo\beta_ala_study\Beta_Ala_Dataframe.csv",
                sep=';')
df.drop('Unnamed: 7', axis=1, inplace=True)
df.dropna(inplace=True)
df

,id,group,age,sex,weight_kg,hight_m,bmi
0,25778.0,Beta Ala,25.0,w,57.0,1.63,21.45
1,54145.0,Beta Ala,21.0,w,72.0,1.72,24.34
2,15073.0,Beta Ala,24.0,m,69.0,1.82,20.38
3,20438.0,Beta Ala,25.0,m,76.0,1.80,23.46
4,49830.0,Beta Ala,26.0,w,82.0,1.68,29.05
5,58046.0,Beta Ala,23.0,m,77.0,1.85,22.50
6,26107.0,Beta Ala,22.0,w,65.0,1.70,22.49
7,44858.0,Beta Ala,26.0,w,53.0,1.64,19.71
8,59289.0,Beta Ala,21.0,m,75.0,1.78,23.67
9,93672.0,Beta Ala,24.0,m,82.0,1.77,26.17


In [6]:
# number of male and female participants
df['sex'].value_counts()

sex
w    12
m    12
Name: count, dtype: int64

In [7]:
# sample by group
frame_by_group = df.groupby('group')
group_statistics = frame_by_group[['age', 'weight_kg', 'hight_m', 'bmi']].agg(['mean', 'std', 'min', 'max'])

pd.set_option('display.float_format', lambda x: '%.2f' % x)

group_statistics = group_statistics.transpose()
group_statistics.index.name = 'Variable'

print('Descriptive Statistics for Each Group\n')
print(group_statistics.to_string())

Descriptive Statistics for Each Group

group           Beta Ala  Placebo
age       mean     24.25    25.58
          std       2.49     2.71
          min      21.00    22.00
          max      30.00    30.00
weight_kg mean     71.83    68.67
          std      11.02     9.92
          min      53.00    58.00
          max      91.00    87.00
hight_m   mean      1.75     1.72
          std       0.10     0.09
          min       1.62     1.54
          max       1.96     1.87
bmi       mean     23.41    22.97
          std       2.51     1.52
          min      19.71    21.05
          max      29.05    25.47


## Import isokinetic raw data from github repository

In [9]:
iso = pd.read_csv(r"C:\Users\bvenn\OneDrive\Desktop\Publikation\Repository\new_repo\beta_ala_study\isokinet_dataframe.csv")

# format data
iso['id'] = iso['id'].astype('int64')
iso['measurement'] = iso['measurement'].astype('category')
iso['experiment'] = iso['experiment'].astype('category')
iso['group'] = iso['group'].astype('category')

# tranform data to its units
iso['pos_1/10_deg/s_or_mm'] = iso['pos_1/10_deg/s_or_mm'] / 10
iso['torque_1/10nm_or_force_n'] = iso['torque_1/10nm_or_force_n'] / (-10)
iso['speed_1/10_deg/sec_or_1/10_mm/s'] = iso['speed_1/10_deg/sec_or_1/10_mm/s'] / 10


iso.rename(columns={'pos_1/10_deg/s_or_mm': 'pos_deg_or_mm', 'torque_1/10nm_or_force_n': 'torque_nm',
                   'speed_1/10_deg/sec_or_1/10_mm/s': 'speed_deg/s_or_mm/s'}, inplace=True)
iso.head()

,id,measurement,time_ms,pos_deg_or_mm,torque_nm,speed_deg/s_or_mm/s,torque_without_comp,rep,set,torque_dyn,force_right,force_left,experiment,group
0,15073,warmup,5,12.00,-3.00,0.00,293,1,1,30,0,0,pretest,beta_ala
1,15073,warmup,10,12.00,-3.00,0.50,293,1,1,30,0,0,pretest,beta_ala
2,15073,warmup,15,12.00,-2.30,4.00,293,1,1,23,0,0,pretest,beta_ala
3,15073,warmup,20,12.10,-0.00,18.30,293,1,1,0,0,0,pretest,beta_ala
4,15073,warmup,25,12.10,11.30,23.50,293,1,1,-113,0,0,pretest,beta_ala


## Calculate Peak Torque for each participant in pre- and post-test
- peak torque maximum torque acomplished
- colum abbruchkriterium = 50% of maximal torque value. At three consequtive repetitions below that value, sufficient fatigue was induced and the protocol was terminated. pre = pretest, post = posttest

In [7]:
# create dataframe with torque values
pt_results = pd.DataFrame(columns = ['id', 'group', 'peak_torque_pre', 'abbruchkriterium_pre', 'peak_torque_post',
                                     'abbruchkriterium_post'])

probanden_ids = iso['id'].unique()

for proband_id in probanden_ids:
    proband_data = iso[iso['id'] == proband_id]
    
    # add group information
    group = proband_data['group'].iloc[0]
    
    # filter for pre and post measurements
    proband_data_pre = proband_data[proband_data['experiment'] == 'pretest']
    proband_data_post = proband_data[proband_data['experiment'] == 'posttest']
    
    # filter for concentric reps. every second rep
    kon_pre = proband_data_pre.query('measurement == "pt" and rep % 2 == 0')
    kon_post = proband_data_post.query('measurement == "pt" and rep % 2 == 0')
    
    # calculate termination criterion
    # pretest
    
    peak_torque_pre = kon_pre['torque_nm'].max()
    abort_pre = peak_torque_pre / 2
    
    # posttest
    
    peak_torque_post = kon_post['torque_nm'].max()

    # termination at 50% max torque
    abort_post = peak_torque_post / 2
    
    ergebnis_row = pd.DataFrame({
        'id': [proband_id],
        'group': [group],
        'peak_torque_pre': [peak_torque_pre],
        'abbruchkriterium_pre': [abort_pre],
        'peak_torque_post': [peak_torque_post],
        'abbruchkriterium_post': [abort_post]
        
    })
    
    # add results to dataframe
    pt_results = pd.concat([pt_results, ergebnis_row], ignore_index = True)
    
pt_results

C:\Users\bvenn\AppData\Local\Temp\ipykernel_11308\2043861747.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pt_results = pd.concat([pt_results, ergebnis_row], ignore_index = True)


,id,group,peak_torque_pre,abbruchkriterium_pre,peak_torque_post,abbruchkriterium_post
0,15073,beta_ala,249.50,124.75,269.00,134.50
1,20438,beta_ala,306.80,153.40,291.00,145.50
2,25778,beta_ala,163.20,81.60,155.70,77.85
3,26107,beta_ala,201.50,100.75,194.00,97.00
4,27351,beta_ala,162.80,81.40,166.50,83.25
5,28514,placebo,121.20,60.60,139.20,69.60
6,93672,beta_ala,294.50,147.25,338.30,169.15
7,33468,placebo,156.50,78.25,171.00,85.50
8,39337,placebo,296.70,148.35,261.50,130.75
9,39750,placebo,139.20,69.60,131.30,65.65


In [8]:
pt_results_l = pd.melt(pt_results.drop(columns=['abbruchkriterium_pre', 'abbruchkriterium_post']), id_vars = ['id', 'group'], value_name='torque', var_name = 'measurement')
pt_results_l.head()

,id,group,measurement,torque
0,15073,beta_ala,peak_torque_pre,249.50
1,20438,beta_ala,peak_torque_pre,306.80
2,25778,beta_ala,peak_torque_pre,163.20
3,26107,beta_ala,peak_torque_pre,201.50
4,27351,beta_ala,peak_torque_pre,162.80


In [9]:
# Peak Torque group differences

(
    ggplot(pt_results_l, aes(x='measurement', y='torque'))
    + geom_boxplot(aes(group='group', color='group'))
    + labs(title='Peak Torque at pre and post test - seperated by groups.')
    + ylab('Peak Torque [Nm]')
)

In [10]:
pt_stat = pg.mixed_anova(dv='torque',between='group', within='measurement', subject='id', data=pt_results_l, correction='auto', effsize='np2')
pt_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,13313.34,1,22,13313.34,1.68,0.21,0.07,NaN
1,measurement,189.61,1,22,189.61,1.12,0.30,0.05,1.00
2,Interaction,23.52,1,22,23.52,0.14,0.71,0.01,NaN


### Numer of Reps done to termination criterion

In [10]:
# Sets done = Number of sets done last set included
# total_reps = total sum of reps


absolvierte_reps = pd.read_csv(r"C:\Users\bvenn\OneDrive\Desktop\Publikation\Repository\new_repo\beta_ala_study\Absolvierte_reps.csv",
                                sep=';')
absolvierte_reps = absolvierte_reps.iloc[:24, :17]
absolvierte_reps.drop('Unnamed: 8', axis=1, inplace=True)
absolvierte_reps

,id,group,sets_done_pre,reps_done_last set_pre,total_reps_pre,sets_done_post,reps_done_last_set_post,total_reps_post,letzte_peaks_1_pre,letzte_peaks_2_pre,letzte_peaks_3_pre,Mean_last_peaks_pre,letzte_peaks_1_post,letzte_peaks_2_post,letzte_peaks_3_post,Mean_last_peaks_post
0,15073.00,beta_ala,5.00,15.00,75.00,6.00,13.00,88.00,127.50,122.30,119.30,123.03,125.70,123.80,115.50,121.67
1,20438.00,beta_ala,2.00,12.00,27.00,7.00,13.00,103.00,146.70,146.30,143.70,145.57,140.00,122.30,123.00,128.43
2,25778.00,beta_ala,4.00,15.00,60.00,2.00,14.00,29.00,75.80,77.00,80.70,77.83,70.50,68.70,62.30,67.17
3,26107.00,beta_ala,8.00,15.00,120.00,6.00,14.00,89.00,101.70,93.80,87.00,94.17,95.00,94.50,87.00,92.17
4,27351.00,beta_ala,2.00,14.00,29.00,1.00,14.00,14.00,56.30,75.00,72.80,68.03,74.30,71.30,65.00,70.20
5,28514.00,placebo,7.00,15.00,105.00,7.00,9.00,99.00,57.80,63.80,65.70,62.43,57.50,53.70,62.30,57.83
6,33468.00,placebo,3.00,15.00,45.00,2.00,14.00,29.00,70.50,74.30,72.80,72.53,81.80,75.80,80.30,79.30
7,39337.00,placebo,3.00,14.00,44.00,4.00,15.00,60.00,149.30,146.30,141.50,145.70,109.50,117.50,117.50,114.83
8,39750.00,placebo,2.00,15.00,30.00,4.00,12.00,57.00,71.30,65.30,60.50,65.70,71.70,56.70,60.50,62.97
9,44858.00,beta_ala,3.00,14.00,44.00,4.00,13.00,58.00,72.00,63.80,61.50,65.77,80.00,68.70,73.50,74.07


In [12]:

absolvierte_reps_l = pd.melt(absolvierte_reps[['total_reps_pre', 'total_reps_post', 'id', 'group']], 
                             id_vars=['id', 'group'], var_name = 'measurement', value_name='reps done')

In [13]:
(
    ggplot(absolvierte_reps_l, aes(x='measurement', y='reps done'))
    + geom_boxplot(aes(color='group', group='group'))
    + labs(title='Completed reps until termation')
    + ylab('Reps total')
)

In [12]:
# statistic of total reps done

rep_stat = pd.read_csv(r"C:\Users\bvenn\OneDrive\Desktop\Publikation\Repository\new_repo\beta_ala_study\Absolvierte_reps_sheet3.csv", 
                         sep = ';')

reps_stat = pg.mixed_anova(dv='reps',between='group', within='experiment', subject='id', data=rep_stat, correction='auto', effsize='np2')
reps_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,784.08,1,22,784.08,0.74,0.40,0.03,NaN
1,experiment,2.08,1,22,2.08,0.01,0.94,0.00,1.00
2,Interaction,108.00,1,22,108.00,0.32,0.58,0.01,NaN


## Calculate work done in pre and post test as integral over the complete range of motion.


In [52]:
iso = pd.read_csv(r"C:\Users\bvenn\OneDrive\Desktop\Publikation\Zusammenfassung Arbeitssätze\isokinet csv\isokinet_dataframe.csv")

# format data
iso['id'] = iso['id'].astype('category')
iso['measurement'] = iso['measurement'].astype('category')
iso['experiment'] = iso['experiment'].astype('category')
iso['group'] = iso['group'].astype('category')

# tranform data to its units
# pos mm to m
iso['pos_m'] = iso['pos_1/10_deg/s_or_mm'] / 1000
# 1/10 Nm to Nm in absolute numbers
iso['torque_nm'] = iso['torque_1/10nm_or_force_n'] / (10)
iso['torque_nm'] = iso['torque_nm'].abs()
# force in Newton
iso['force_n'] = iso['torque_1/10nm_or_force_n']
# displacement between two time points
iso['displacement'] = iso['pos_m'].diff().abs()
# culumative distance
iso['cumulative_distance'] = iso['displacement'].cumsum()

iso.drop(['time_ms', 'pos_1/10_deg/s_or_mm', 'torque_1/10nm_or_force_n', 'speed_1/10_deg/sec_or_1/10_mm/s',
          'torque_without_comp', 'rep', 'set', 'torque_dyn', 'force_right', 'force_left'], axis=1, inplace=True)
iso

,id,measurement,experiment,group,pos_m,torque_nm,force_n,displacement,cumulative_distance
0,15073,warmup,pretest,beta_ala,0.120,3.0,30,NaN,NaN
1,15073,warmup,pretest,beta_ala,0.120,3.0,30,0.000,0.000
2,15073,warmup,pretest,beta_ala,0.120,2.3,23,0.000,0.000
3,15073,warmup,pretest,beta_ala,0.121,0.0,0,0.001,0.001
4,15073,warmup,pretest,beta_ala,0.121,11.3,-113,0.000,0.001
...,...,...,...,...,...,...,...,...,...
2267524,93672,s2,posttest,beta_ala,0.157,22.5,-225,0.003,5783.799
2267525,93672,s2,posttest,beta_ala,0.155,22.5,-225,0.002,5783.801
2267526,93672,s2,posttest,beta_ala,0.151,22.2,-222,0.004,5783.805
2267527,93672,s2,posttest,beta_ala,0.150,21.8,-218,0.001,5783.806


In [65]:
# extract ids
probanden_ids = iso['id'].unique()

# create data-frame
total_work_done = pd.DataFrame(columns=['id', 'experiment', 'group', 'work_done'])

for proband_id in probanden_ids:
    proband_data = iso[iso['id'] == proband_id]
    
    # group information
    group = proband_data['group'].iloc[0]
    
    # filter for pre and post-test
    proband_data_pre = proband_data[proband_data['experiment'] == 'pretest'].copy()
    proband_data_post = proband_data[proband_data['experiment'] == 'posttest'].copy()
    
    # calculate work done for pretest
    if not proband_data_pre.empty:
        proband_data_pre = proband_data_pre.dropna(subset=['displacement', 'cumulative_distance'])
        work_done_pre = trapz(proband_data_pre['torque_nm'], proband_data_pre['cumulative_distance'])
    else:
        work_done_pre = np.nan
    
    # calculate work done for posttest
    if not proband_data_post.empty:
        proband_data_post = proband_data_post.dropna(subset=['displacement', 'cumulative_distance'])
        work_done_post = trapz(proband_data_post['torque_nm'], proband_data_post['cumulative_distance'])
    else:
        work_done_post = np.nan
    
    # append results to total_work_done DataFrame
    total_work_done = pd.concat([total_work_done, pd.DataFrame({'id': [proband_id, proband_id],
                                                               'experiment': ['pretest', 'posttest'],
                                                               'group': [group, group],
                                                               'work_done': [work_done_pre, work_done_post]})], ignore_index=True)


C:\Users\bvenn\AppData\Local\Temp\ipykernel_11328\502453469.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_work_done = pd.concat([total_work_done, pd.DataFrame({'id': [proband_id, proband_id],


In [66]:
total_work_done['work_done'].max()

21250.230149994743

In [67]:
work_pre = total_work_done[total_work_done['experiment']=='pretest'].reset_index(drop=True)
work_post = total_work_done[total_work_done['experiment']=='posttest'].reset_index(drop=True)

In [68]:
work_pre['work_diff'] = work_post['work_done'] - work_pre['work_done']
work_pre.drop(['experiment', 'work_done'], axis=1, inplace=True)
work_pre

,id,group,work_diff
0,15073,beta_ala,2037.772550
1,20438,beta_ala,9622.332600
2,25778,beta_ala,-4748.549000
3,26107,beta_ala,-3717.778350
4,27351,beta_ala,-1333.338000
5,28514,placebo,511.294700
6,93672,beta_ala,203.740250
7,33468,placebo,-1556.626550
8,39337,placebo,2164.645100
9,39750,placebo,2426.897850


In [71]:
(
    ggplot(work_pre, aes(x='group', y='work_diff', color='group'))
    + geom_boxplot()
    + labs(title='Changes in total work done from prtest to posttest')
    + ylab('Work Difference [J]')
)

In [72]:
x = work_pre.query('group == "placebo"')['work_diff']
y = work_pre.query('group == "beta_ala"')['work_diff']

In [73]:
work_stat = pg.ttest(x, y, paired=False, alternative='less', correction='auto')
work_stat

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.419159,22,less,0.339582,"[-inf, 1752.18]",0.171121,0.796,0.107774


# Determine fatigue index from set one to termination

In [46]:
s1_abbruch

,id,measurement,time_ms,pos_deg_or_mm,torque_nm,speed_deg/s_or_mm/s,torque_without_comp,rep,set,torque_dyn,force_right,force_left,experiment,group
0,15073,s1,5,11.40,21.80,0.00,72,1,1,-218,0,0,pretest,beta_ala
1,15073,s1,10,11.60,21.80,0.30,68,1,1,-218,0,0,pretest,beta_ala
2,15073,s1,15,11.60,22.50,1.80,65,1,1,-225,0,0,pretest,beta_ala
3,15073,s1,20,11.40,24.00,4.30,57,1,1,-240,0,0,pretest,beta_ala
4,15073,s1,25,11.40,26.70,5.80,53,1,1,-267,0,0,pretest,beta_ala
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1473081,93672,s2,43210,11.30,15.50,-15.30,150,28,1,-155,0,0,posttest,beta_ala
1473082,93672,s2,43215,11.10,20.30,-10.00,102,28,1,-203,0,0,posttest,beta_ala
1473083,93672,s2,43220,11.10,25.20,-8.30,53,28,1,-252,0,0,posttest,beta_ala
1473084,93672,s2,43225,11.10,30.00,-8.00,5,28,1,-300,0,0,posttest,beta_ala


## mean of top thee torque peaks

In [47]:
peaks_largest = []

for _, sub_df in iso.groupby(['id', 'experiment', 'group']):
    proband_id, experiment_type, group = _
    
    px = sub_df.query('rep % 2 == 0').copy()
    
    px['torque_nm_abs'] = px['torque_nm'].abs()
    px.loc[:, 'time_ms'] = px.index * 5 + 5
    peaks, properties = find_peaks(px['torque_nm_abs'], height=50, distance=200)
    top_three_peaks = px['torque_nm_abs'].iloc[peaks].nlargest(3)
    mean_top_three = top_three_peaks.mean()
    
    
    # save results
    peaks_largest.append({
        'id': proband_id,
        'experiment': experiment_type,
        'group': group,
        'top_three_peaks': top_three_peaks.tolist(),
        'mean_top_three': mean_top_three,
    })

# to dataframe
peaks_largest = pd.DataFrame(peaks_largest)



C:\Users\bvenn\AppData\Local\Temp\ipykernel_11308\3135058859.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for _, sub_df in iso.groupby(['id', 'experiment', 'group']):


## mean of peaks from last three reps

In [13]:
# to open the file download the Absolvierte_reps.xlsx file and copy its path within the brackets
means_last_reps = pd.read_csv(r"C:\Users\bvenn\OneDrive\Desktop\Publikation\Repository\new_repo\beta_ala_study\Absolvierte_reps_sheet2.csv",
                                sep=';')
means_last_reps

,id,group,experiment,letzte_peaks_1,letzte_peaks_2,letzte_peaks_3,Mean_peaks
0,15073,beta_ala,pretest,127.50,122.30,119.30,123.03
1,20438,beta_ala,pretest,146.70,146.30,143.70,145.57
2,25778,beta_ala,pretest,75.80,77.00,80.70,77.83
3,26107,beta_ala,pretest,101.70,93.80,87.00,94.17
4,27351,beta_ala,pretest,56.30,75.00,72.80,68.03
5,28514,placebo,pretest,57.80,63.80,65.70,62.43
6,33468,placebo,pretest,70.50,74.30,72.80,72.53
7,39337,placebo,pretest,149.30,146.30,141.50,145.70
8,39750,placebo,pretest,71.30,65.30,60.50,65.70
9,44858,beta_ala,pretest,72.00,63.80,61.50,65.77


In [49]:
peaks_largest = pd.merge(peaks_largest, means_last_reps[['experiment', 'id', 'group', 'Mean_peaks']], 
                         on=['experiment', 'id', 'group'], how='left')



### calculate fatigue index

In [50]:
peaks_largest['fatigue_index'] = ((peaks_largest['mean_top_three'] - peaks_largest['Mean_peaks']) / 
                                peaks_largest['mean_top_three']) * 100
peaks_largest

,id,experiment,group,top_three_peaks,mean_top_three,Mean_peaks,fatigue_index
0,15073,posttest,beta_ala,"[269.0, 264.8, 249.0]",260.93,121.67,53.37
1,15073,pretest,beta_ala,"[249.5, 248.0, 243.0]",246.83,123.03,50.16
2,20438,posttest,beta_ala,"[291.0, 284.7, 284.0]",286.57,128.43,55.18
3,20438,pretest,beta_ala,"[306.8, 294.0, 292.2]",297.67,145.57,51.10
4,25778,posttest,beta_ala,"[155.7, 152.3, 147.5]",151.83,67.17,55.76
5,25778,pretest,beta_ala,"[163.2, 154.5, 145.5]",154.40,77.83,49.59
6,26107,posttest,beta_ala,"[195.0, 194.0, 186.5]",191.83,92.17,51.95
7,26107,pretest,beta_ala,"[201.5, 186.8, 182.0]",190.10,94.17,50.46
8,27351,posttest,beta_ala,"[166.5, 161.0, 156.0]",161.17,70.20,56.44
9,27351,pretest,beta_ala,"[162.8, 162.8, 155.0]",160.20,68.03,57.53


## pre to post difference of fatigue index

In [51]:
fatigue_pre = peaks_largest[peaks_largest['experiment'] == 'pretest'].reset_index(drop=True)
fatigue_post = peaks_largest[peaks_largest['experiment'] == 'posttest'].reset_index(drop=True)

In [52]:
fatigue_pre['diff_fatigue_index'] = fatigue_post['fatigue_index'] - fatigue_pre['fatigue_index']
fatigue_pre

,id,experiment,group,top_three_peaks,mean_top_three,Mean_peaks,fatigue_index,diff_fatigue_index
0,15073,pretest,beta_ala,"[249.5, 248.0, 243.0]",246.83,123.03,50.16,3.21
1,20438,pretest,beta_ala,"[306.8, 294.0, 292.2]",297.67,145.57,51.10,4.09
2,25778,pretest,beta_ala,"[163.2, 154.5, 145.5]",154.40,77.83,49.59,6.17
3,26107,pretest,beta_ala,"[201.5, 186.8, 182.0]",190.10,94.17,50.46,1.49
4,27351,pretest,beta_ala,"[162.8, 162.8, 155.0]",160.20,68.03,57.53,-1.09
5,28514,pretest,placebo,"[121.2, 119.0, 118.2]",119.47,62.43,47.74,8.18
6,33468,pretest,placebo,"[158.0, 157.5, 156.5]",157.33,72.53,53.90,-2.68
7,39337,pretest,placebo,"[296.7, 274.2, 268.5]",279.80,145.70,47.93,7.41
8,39750,pretest,placebo,"[139.2, 135.5, 131.7]",135.47,65.70,51.50,-0.06
9,44858,pretest,beta_ala,"[143.3, 142.2, 140.7]",142.07,65.77,53.70,-3.61


In [53]:
(
    ggplot(fatigue_pre, aes(x='group', y='diff_fatigue_index', color='group'))
    + geom_boxplot()
    + ylab('Difference of Fatigue Index')
)

In [54]:
x = fatigue_pre.query('group == "placebo"')['diff_fatigue_index']
y = fatigue_pre.query('group == "beta_ala"')['diff_fatigue_index']

work_stat = pg.ttest(x, y, paired=False, alternative='less', correction='auto')
work_stat

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.07,22,less,0.85,"[-inf, 5.67]",0.44,0.884,0.00


# TMG Analysis

In [2]:
tmg_raw = pd.read_csv(r"C:\Users\bvenn\OneDrive\Desktop\Publikation\Repository\beta_ala_study\tmg_raw_data.csv")
tmg_raw.rename(columns={'Unnamed: 0': 'index', 'value': 'displacement'}, inplace=True)

# adjust datatypes
tmg_raw = tmg_raw.astype({'group': 'string', 'experiment': 'string', 'measurement': 'string'})

# add time
millisekenden_pro_datenpunkt = 1
tmg_raw['time_ms'] = tmg_raw.groupby(['id', 'experiment', 'measurement'])['index'].transform(lambda x: (x - x.min()) * millisekenden_pro_datenpunkt)

tmg_raw

,index,id,group,experiment,measurement,displacement,time_ms
0,0,15073,beta_ala,posttest,supine_1,0.000000,0
1,1,15073,beta_ala,posttest,supine_1,-0.000108,1
2,2,15073,beta_ala,posttest,supine_1,-0.000600,2
3,3,15073,beta_ala,posttest,supine_1,-0.001181,3
4,4,15073,beta_ala,posttest,supine_1,-0.001808,4
...,...,...,...,...,...,...,...
500995,501995,93672,beta_ala,pretest,supine_4,0.008989,995
500996,501996,93672,beta_ala,pretest,supine_4,0.004278,996
500997,501997,93672,beta_ala,pretest,supine_4,0.001684,997
500998,501998,93672,beta_ala,pretest,supine_4,0.000508,998


In [32]:
example_data = tmg_raw.query('id == 20438 and experiment == "posttest" and measurement == "supine_1"')

# threshold for peaks Peaks
threshold = 0.04

# Wert des ersten Peaks der Kurve für dieses spezifische Beispiel
peaks, _ = find_peaks(example_data['displacement'].values, height=threshold)
dm = example_data['displacement'].iloc[peaks[0]] if len(peaks) > 0 else np.nan
time_of_dm = example_data['time_ms'].iloc[peaks[0]] if len(peaks) > 0 else np.nan

# Td berechnen: Schwellenwert für Td
threshold_td = 0.1 * dm
# Index von Td berechnen
index_td = example_data.index[example_data['displacement'] >= threshold_td].min()
# Zeit für Td berechnen
time_td = example_data.loc[index_td, 'time_ms'] if pd.notna(index_td) else np.nan

# Schwellenwert für Tc
threshold_tc = 0.9 * dm
# Index von Td berechnen
index_tc = example_data.index[example_data['displacement'] >= threshold_tc].min()
# Zeit für Td berechnen
time_tc = example_data.loc[index_tc, 'time_ms'] if pd.notna(index_tc) else np.nan


# Finde den ersten Wert, der 50% von dm entspricht
threshold_ts = 0.5 * dm
erster_wert_index = None

for i, wert in enumerate(example_data['displacement'].values):
    if wert >= threshold_ts:
        erster_wert_index = i
        break

# Iteriere weiter ab diesem Wert und finde den nächsten Wert, der 50% von dm entspricht
zweiter_wert_index = None

if erster_wert_index is not None:
    for y, wert in enumerate(example_data['displacement'].values[erster_wert_index + 1:]):
        if wert <= threshold_ts:
            zweiter_wert_index = erster_wert_index + 1 + y
            break

# Überprüfe, ob die Indizes gültig sind
if erster_wert_index is not None and zweiter_wert_index is not None:
    erster_wert_index += example_data.index[0]
    zweiter_wert_index += example_data.index[0]

# Berechne die Zeiten für ts_start und ts_end
time_ts_start = example_data.loc[erster_wert_index, 'time_ms'] if erster_wert_index is not None else np.nan
time_ts_end = example_data.loc[zweiter_wert_index, 'time_ms'] if zweiter_wert_index is not None else np.nan

# Finde den ersten Wert, der 90% von dm entspricht
threshold_tr = 0.9 * dm
erste_zahl_index = None

for i, zahl in enumerate(example_data['displacement'].values):
    if zahl >= threshold_tr:
        erste_zahl_index = i
        break

# Iteriere weiter ab diesem Wert und finde den nächsten Wert, der 90% von dm entspricht
zweite_zahl_index = None

if erste_zahl_index is not None:
    for y, zahl in enumerate(example_data['displacement'].values[erste_zahl_index + 1:]):
        if zahl <= threshold_tr:
            zweite_zahl_index = erste_zahl_index + 1 + y
            break

# Überprüfe, ob die Indizes gültig sind
if zweite_zahl_index is not None and zweite_zahl_index is not None:
    erste_zahl_index += example_data.index[0]
    zweite_zahl_index += example_data.index[0]
    
index_tr_end = example_data.index[example_data['time_ms'] == time_ts_end].min()
    
time_tr_start = example_data.loc[zweite_zahl_index, 'time_ms'] if zweite_zahl_index is not None else np.nan
time_tr_end = example_data.loc[index_tr_end, 'time_ms'] if index_tr_end is not None else np.nan

print("Dm:", dm, 'mm')
print("Td:", time_td, 'ms')
print("Tc:", time_tc - time_td, 'ms')
print("Ts:", time_ts_end - time_ts_start, 'ms')
print('Tr:', time_tr_end - time_tr_start, 'ms')
print('Vc:', (dm / (time_td + time_tc)), 'mm/ms')


# Deine ggplot-Grafik bleibt unverändert, füge einfach die Punkte für ts_start und ts_end hinzu
(
    ggplot(example_data, aes(x='time_ms', y='displacement'))
    + geom_line()
    + geom_point(data=example_data.iloc[[peaks[0]]], mapping=aes(x='time_ms', y='displacement'), color='red', size=3)
    + geom_point(aes(x='time_tc', y='displacement'), data=pd.DataFrame({'time_tc': [time_tc], 'displacement': [dm * 0.9]}), color='green', size=3)
    + geom_point(aes(x='time_td', y='displacement'), data=pd.DataFrame({'time_td': [time_td], 'displacement': [dm * 0.1]}), color='blue', size=3)
    + geom_point(aes(x='time_ts_start', y='displacement'), data=pd.DataFrame({'time_ts_start': [time_ts_start], 'displacement': [dm * 0.5]}), color='orange', size=3)
    + geom_point(aes(x='time_ts_end', y='displacement'), data=pd.DataFrame({'time_ts_end': [time_ts_end], 'displacement': [dm * 0.5]}), color='orange', size=3)
    + geom_point(aes(x='time_tr_start', y='displacement'), data=pd.DataFrame({'time_tr_start': [time_tr_start], 'displacement': [dm * 0.9]}), color='black', size=3)
    + geom_point(aes(x='time_tr_end', y='displacement'), data=pd.DataFrame({'time_tr_end': [time_tr_end], 'displacement': [dm * 0.9]}), color='black', size=3)
)


Dm: 8.59960565326745 mm
Td: 27 ms
Tc: 25 ms
Ts: 49 ms
Tr: 21 ms
Vc: 0.10885576776287911 mm/ms


# Calculte individual tmg parameters for each measurement

In [3]:
# create dataframe
tmg_parameter = pd.DataFrame(columns=['id', 'group', 'experiment', 'measurement', 'Dm', 'Td', 'Tc', 'Ts', 'Tr', 'Vc'])

# threshold for Peaks
threshold = 0.04

# iterate by ID
for unique_id in tmg_raw['id'].unique():
    # for pre and post test
    for unique_experiment in tmg_raw['experiment'].unique():
        # measurement time point
        for unique_measurement in tmg_raw['measurement'].unique():
            # filter data
            subset = tmg_raw[(tmg_raw['id'] == unique_id) & 
                             (tmg_raw['experiment'] == unique_experiment) & 
                             (tmg_raw['measurement'] == unique_measurement)].copy()
            
            if not subset.empty:
                parameter_value = subset['displacement'].mean()
                
                # group
                group = subset['group'].iloc[0]

                # value of first peak
                peaks, _ = find_peaks(subset['displacement'].values, height=threshold)
                dm = subset['displacement'].iloc[peaks[0]] if len(peaks) > 0 else np.nan

                # Td threshold
                threshold_td = 0.1 * dm
                index_td = subset.index[subset['displacement'] >= threshold_td].min()
                # Td time
                time_td = subset.loc[index_td, 'time_ms'] if pd.notna(index_td) and pd.notna(subset.loc[index_td, 'time_ms']) else np.nan

                # threshold Tc
                threshold_tc = 0.9 * dm
                index_tc = subset.index[subset['displacement'] >= threshold_tc].min()
                # time tc
                time_tc = subset.loc[index_tc, 'time_ms'] if pd.notna(index_tc) and pd.notna(subset.loc[index_tc, 'time_ms']) else np.nan

                # 50% dm
                threshold_ts = 0.5 * dm
                erster_wert_index = None

                for i, wert in enumerate(subset['displacement'].values):
                    if wert >= threshold_ts:
                        erster_wert_index = i
                        break

                # next 50% dm
                zweiter_wert_index = None

                if erster_wert_index is not None:
                    for y, wert in enumerate(subset['displacement'].values[erster_wert_index + 1:]):
                        if wert <= threshold_ts:
                            zweiter_wert_index = erster_wert_index + 1 + y
                            break

                if erster_wert_index is not None and zweiter_wert_index is not None:
                    erster_wert_index += subset.index[0]
                    zweiter_wert_index += subset.index[0]

                # time ts start and end
                time_ts_start = subset.loc[erster_wert_index, 'time_ms'] if pd.notna(erster_wert_index) and pd.notna(subset.loc[erster_wert_index, 'time_ms']) else np.nan
                time_ts_end = subset.loc[zweiter_wert_index, 'time_ms'] if pd.notna(zweiter_wert_index) and pd.notna(subset.loc[zweiter_wert_index, 'time_ms']) else np.nan

                # 90% dm
                threshold_tr = 0.9 * dm
                erste_zahl_index = None

                for i, zahl in enumerate(subset['displacement'].values):
                    if zahl >= threshold_tr:
                        erste_zahl_index = i
                        break

                # next 90% dm
                zweite_zahl_index = None

                if erste_zahl_index is not None:
                    for y, zahl in enumerate(subset['displacement'].values[erste_zahl_index + 1:]):
                        if zahl <= threshold_tr:
                            zweite_zahl_index = erste_zahl_index + 1 + y
                            break

                if zweite_zahl_index is not None and zweite_zahl_index is not None:
                    erste_zahl_index += subset.index[0]
                    zweite_zahl_index += subset.index[0]

                index_tr_end = subset.index[subset['time_ms'] == time_ts_end].min()

                # time tr start and end
                time_tr_start = subset.loc[zweite_zahl_index, 'time_ms'] if pd.notna(zweite_zahl_index) and pd.notna(subset.loc[zweite_zahl_index, 'time_ms']) else np.nan
                time_tr_end = subset.loc[index_tr_end, 'time_ms'] if pd.notna(index_tr_end) and pd.notna(subset.loc[index_tr_end, 'time_ms']) else np.nan

                tmg_parameter = pd.concat([tmg_parameter, pd.DataFrame({
                    'id': [unique_id],
                    'group':[group],
                    'experiment': [unique_experiment],
                    'measurement': [unique_measurement],
                    'Dm': [dm],
                    'Td': [time_td],
                    'Tc': [time_tc - time_td],
                    'Ts': [time_ts_end - time_ts_start],
                    'Tr': [time_tr_end - time_tr_start],
                    'Vc': [dm / (time_td + time_tc)]
                })], ignore_index=True)

tmg_parameter


C:\Users\bvenn\AppData\Local\Temp\ipykernel_11328\2537145000.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tmg_parameter = pd.concat([tmg_parameter, pd.DataFrame({


,id,group,experiment,measurement,Dm,Td,Tc,Ts,Tr,Vc
0,15073,beta_ala,posttest,supine_1,9.949456,31,25,77,46,0.114362
1,15073,beta_ala,posttest,before_warmup,2.989571,23,26,179,146,0.041522
2,15073,beta_ala,posttest,after_warmup,2.272929,20,23,194,36,0.036078
3,15073,beta_ala,posttest,after_pt,1.812468,19,21,167,138,0.030720
4,15073,beta_ala,posttest,s1,0.742931,16,17,26,7,0.015162
...,...,...,...,...,...,...,...,...,...,...
496,93672,beta_ala,pretest,s1,3.199221,22,19,28,8,0.050781
497,93672,beta_ala,pretest,s2,2.277724,24,19,30,9,0.033996
498,93672,beta_ala,pretest,supine_2,8.410425,25,26,137,103,0.110663
499,93672,beta_ala,pretest,supine_3,11.591811,25,29,144,111,0.146732


In [7]:
#tmg_parameter.to_excel(r"C:\Users\bvenn\OneDrive\Desktop\Publikation\Repository\beta_ala_study\tmg_parameter.xlsx")

In [4]:
tmg_parameter = tmg_parameter.astype({'group': 'category', 'experiment': 'category', 'measurement': 'category', 
                                       'Td': 'float64', 'Tc': 'float64', 'Dm': 'float64', 'Ts': 'float64', 'Tr': 'float64',
                                      'Vc': 'float64'})

tmg_parameter['experiment'] = pd.Categorical(tmg_parameter['experiment'], categories=['pretest', 'posttest'], ordered=True)

tmg_parameter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   id           501 non-null    object  
 1   group        501 non-null    category
 2   experiment   501 non-null    category
 3   measurement  501 non-null    category
 4   Dm           501 non-null    float64 
 5   Td           501 non-null    float64 
 6   Tc           501 non-null    float64 
 7   Ts           501 non-null    float64 
 8   Tr           501 non-null    float64 
 9   Vc           501 non-null    float64 
dtypes: category(3), float64(6), object(1)
memory usage: 29.9+ KB


In [5]:
# adding pre or posttest
tmg_parameter['measurement'] = tmg_parameter.apply(lambda row: row['measurement']
                                                   + '_pre' if 'pretest' in row['experiment'] else row['measurement'], axis=1)
tmg_parameter['measurement'] = tmg_parameter.apply(lambda row: row['measurement']
                                                   + '_post' if 'posttest' in row['experiment'] else row['measurement'], axis=1)

In [6]:
tmg_parameter.head(-1)

,id,group,experiment,measurement,Dm,Td,Tc,Ts,Tr,Vc
0,15073,beta_ala,posttest,supine_1_post,9.949456,31.0,25.0,77.0,46.0,0.114362
1,15073,beta_ala,posttest,before_warmup_post,2.989571,23.0,26.0,179.0,146.0,0.041522
2,15073,beta_ala,posttest,after_warmup_post,2.272929,20.0,23.0,194.0,36.0,0.036078
3,15073,beta_ala,posttest,after_pt_post,1.812468,19.0,21.0,167.0,138.0,0.030720
4,15073,beta_ala,posttest,s1_post,0.742931,16.0,17.0,26.0,7.0,0.015162
...,...,...,...,...,...,...,...,...,...,...
495,93672,beta_ala,pretest,after_pt_pre,5.181711,20.0,17.0,27.0,9.0,0.090907
496,93672,beta_ala,pretest,s1_pre,3.199221,22.0,19.0,28.0,8.0,0.050781
497,93672,beta_ala,pretest,s2_pre,2.277724,24.0,19.0,30.0,9.0,0.033996
498,93672,beta_ala,pretest,supine_2_pre,8.410425,25.0,26.0,137.0,103.0,0.110663


## New seated anlysis

In [9]:
# new frames for pretest and posttets, all parameter values after warmup
pretest_seated_after_wu = tmg_parameter.loc[(tmg_parameter['experiment'] == 'pretest') & 
                                (tmg_parameter['measurement'] == 'after_warmup_pre')]

posttest_seated_after_wu = tmg_parameter.loc[(tmg_parameter['experiment'] == 'posttest') & 
                                (tmg_parameter['measurement'] == 'after_warmup_post')]


In [12]:
# frames for pretest and posttest containing all set parameter values
pretest_seated_sx = tmg_parameter.loc[(tmg_parameter['experiment'] == 'pretest') & 
                                (tmg_parameter['measurement'].str.contains('s\d'))]

posttest_seated_sx = tmg_parameter.loc[(tmg_parameter['experiment'] == 'posttest') & 
                                (tmg_parameter['measurement'].str.contains('s\d'))]

# frames for last set values
pretest_seated_sx_max_per_id = pretest_seated_sx.groupby('id')['measurement'].max().reset_index()
pretest_seated_sx_max_rows = pretest_seated_sx.merge(pretest_seated_sx_max_per_id, on=['id', 'measurement'], how='inner')

posttest_seated_sx_max_per_id = posttest_seated_sx.groupby('id')['measurement'].max().reset_index()
posttest_seated_sx_max_rows = posttest_seated_sx.merge(posttest_seated_sx_max_per_id, on=['id', 'measurement'], how='inner')

# rest index
pretest_seated_sx_max_rows.reset_index(drop=True, inplace=True)
pretest_seated_after_wu.reset_index(drop=True, inplace=True)

# Reset Index für posttest_seated_sx_max_rows und posttest_seated_after_wu
posttest_seated_sx_max_rows.reset_index(drop=True, inplace=True)
posttest_seated_after_wu.reset_index(drop=True, inplace=True)


In [13]:
# dataframe for differences from after warmup to termination in pretest
pretest_diff = pd.DataFrame()

common_columns = ['id', 'group', 'experiment']

for col in common_columns:
    pretest_diff[col] = pretest_seated_sx_max_rows[col]
    
for param in ['Dm', 'Td', 'Tc', 'Ts', 'Tr', 'Vc']:
    pretest_diff[param] = pretest_seated_sx_max_rows[param] - pretest_seated_after_wu[param]

pretest_diff.head()


,id,group,experiment,Dm,Td,Tc,Ts,Tr,Vc
0,15073,beta_ala,pretest,-1.927345,-1.0,0.0,-167.0,-166.0,-0.033029
1,20438,beta_ala,pretest,-1.325736,0.0,-4.0,-5.0,-1.0,-0.025790
2,25778,beta_ala,pretest,-1.234278,-9.0,5.0,0.0,-1.0,-0.021763
3,26107,beta_ala,pretest,-0.978542,2.0,0.0,-105.0,-103.0,-0.017702
4,27351,beta_ala,pretest,-0.673070,-1.0,0.0,-135.0,-132.0,-0.007789


In [14]:
# dataframe for differences from after warmup to termination in posttest

posttest_diff = pd.DataFrame()
common_columns = ['id', 'group', 'experiment']

for col in common_columns:
    posttest_diff[col] = posttest_seated_sx_max_rows[col]
    
for param in ['Dm', 'Td', 'Tc', 'Ts', 'Tr', 'Vc']:
    posttest_diff[param] = posttest_seated_sx_max_rows[param] - posttest_seated_after_wu[param]
    
posttest_diff.head()

,id,group,experiment,Dm,Td,Tc,Ts,Tr,Vc
0,15073,beta_ala,posttest,-1.594417,1.0,-3.0,-23.0,110.0,-0.025134
1,20438,beta_ala,posttest,-1.236830,1.0,5.0,24.0,10.0,-0.026392
2,25778,beta_ala,posttest,-1.067054,1.0,3.0,2.0,0.0,-0.021753
3,26107,beta_ala,posttest,-0.706058,3.0,-1.0,-202.0,-202.0,-0.013451
4,27351,beta_ala,posttest,-0.525296,0.0,0.0,21.0,26.0,-0.008081


In [15]:
tmg_seated_diffs = pd.concat([pretest_diff, posttest_diff])
tmg_seated_diffs.reset_index(drop=True, inplace=True)

tmg_seated_diffs



,id,group,experiment,Dm,Td,Tc,Ts,Tr,Vc
0,15073,beta_ala,pretest,-1.927345,-1.0,0.0,-167.0,-166.0,-0.033029
1,20438,beta_ala,pretest,-1.325736,0.0,-4.0,-5.0,-1.0,-0.025790
2,25778,beta_ala,pretest,-1.234278,-9.0,5.0,0.0,-1.0,-0.021763
3,26107,beta_ala,pretest,-0.978542,2.0,0.0,-105.0,-103.0,-0.017702
4,27351,beta_ala,pretest,-0.673070,-1.0,0.0,-135.0,-132.0,-0.007789
5,28514,placebo,pretest,-2.696394,-2.0,-6.0,-89.0,-72.0,-0.038459
6,33468,placebo,pretest,1.724444,4.0,24.0,-55.0,-140.0,-0.005704
7,39337,placebo,pretest,-1.614100,4.0,2.0,2.0,0.0,-0.033655
8,39750,placebo,pretest,-2.097513,0.0,0.0,-1.0,-2.0,-0.038843
9,44858,beta_ala,pretest,-0.621867,2.0,2.0,-63.0,-63.0,-0.015609


## diff Dm

In [16]:
(
    ggplot(tmg_seated_diffs, aes(x='experiment', y='Dm', color='group', group='group'))
    + geom_boxplot(width=0.5)
    + labs(title='Differences between after warmup and last measurement')
    + ylab('Dm Difference [mm]')
    + xlab('Measurement')
    + scale_x_discrete(limits=['pretest', 'posttest'], labels=['Pre-Test', 'Post-Test'])
)

In [17]:
dm_stat = pg.mixed_anova(dv='Dm',between='group', within='experiment', subject='id', data=tmg_seated_diffs, correction='auto', effsize='np2')
dm_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,0.090000,1,22,0.090000,0.043583,0.836551,0.001977,NaN
1,experiment,0.051341,1,22,0.051341,0.080825,0.778841,0.003660,1.0
2,Interaction,0.059653,1,22,0.059653,0.093911,0.762144,0.004251,NaN


## Diff Td

In [18]:
(
    ggplot(tmg_seated_diffs, aes(x='experiment', y='Td', color='group', group='group'))
    + geom_boxplot(width=0.5)
    + labs(title='Differences between after warmup and last measurement')
    + ylab('Td Difference [ms]')
    + xlab('measurement')
    + scale_x_discrete(limits=['pretest', 'posttest'], labels=['Pre-Test', 'Post-Test'])
)

In [19]:
Td_stat = pg.mixed_anova(dv='Td',between='group', within='experiment', subject='id', data=tmg_seated_diffs, correction='auto', effsize='np2')
Td_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,2.520833,1,22,2.520833,0.270583,0.608136,0.012150,NaN
1,experiment,4.687500,1,22,4.687500,1.209086,0.283401,0.052095,1.0
2,Interaction,3.520833,1,22,3.520833,0.908158,0.350958,0.039643,NaN


## diff Tc

In [20]:
(
    ggplot(tmg_seated_diffs, aes(x='experiment', y='Tc', color='group', group='group'))
    + geom_boxplot(width=0.5)
    + labs(title='Differences between after warmup and last measurement')
    + ylab('Tc Difference [ms]')
    + xlab('measurement')
    + scale_x_discrete(limits=['pretest', 'posttest'], labels=['Pre-Test', 'Post-Test'])
)

In [21]:
Tc_stat = pg.mixed_anova(dv='Tc',between='group', within='experiment', subject='id', data=tmg_seated_diffs, correction='auto', effsize='np2')
Tc_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,6.750000,1,22,6.750000,0.220599,0.643206,0.009928,NaN
1,experiment,0.083333,1,22,0.083333,0.004562,0.946757,0.000207,1.0
2,Interaction,2.083333,1,22,2.083333,0.114061,0.738769,0.005158,NaN


## diff Ts

In [22]:
(
    ggplot(tmg_seated_diffs, aes(x='experiment', y='Ts', color='group', group='group'))
    + geom_boxplot(width=0.5)
    + labs(title='Differences between after warmup and last measurement')
    + ylab('Ts Difference [ms]')
    + xlab('measurement')
    + scale_x_discrete(limits=['pretest', 'posttest'], labels=['Pre-Test', 'Post-Test'])
)

In [23]:
Ts_stat = pg.mixed_anova(dv='Ts',between='group', within='experiment', subject='id', data=tmg_seated_diffs, correction='auto', effsize='np2')
Ts_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,229.687500,1,22,229.687500,0.021952,0.883563,0.000997,NaN
1,experiment,1485.187500,1,22,1485.187500,0.947096,0.341039,0.041273,1.0
2,Interaction,105.020833,1,22,105.020833,0.066971,0.798205,0.003035,NaN


## diff Tr

In [24]:
(
    ggplot(tmg_seated_diffs, aes(x='experiment', y='Tr', color='group', group='group'))
    + geom_boxplot(width=0.5)
    + labs(title='Differences between after warmup and last measurement')
    + ylab('Tr Difference [ms]')
    + xlab('measurement')
    + scale_x_discrete(limits=['pretest', 'posttest'], labels=['Pre-Test', 'Post-Test'])
)

In [25]:
Tr_stat = pg.mixed_anova(dv='Tr',between='group', within='experiment', subject='id', data=tmg_seated_diffs, correction='auto', effsize='np2')
Tr_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,660.083333,1,22,660.083333,0.062098,0.805523,0.002815,NaN
1,experiment,5808.000000,1,22,5808.000000,1.754691,0.198884,0.073867,1.0
2,Interaction,133.333333,1,22,133.333333,0.040282,0.842774,0.001828,NaN


## diff Vc

In [26]:
(
    ggplot(tmg_seated_diffs, aes(x='experiment', y='Vc', color='group', group='group'))
    + geom_boxplot(width=0.5)
    + labs(title='Differences between after warmup and last measurement')
    + ylab('Vc Difference [mm/ms]')
    + xlab('measurement')
    + scale_x_discrete(limits=['pretest', 'posttest'], labels=['Pre-Test', 'Post-Test'])
)

In [27]:
Vc_stat = pg.mixed_anova(dv='Vc',between='group', within='experiment', subject='id', data=tmg_seated_diffs, correction='auto', effsize='np2')
Vc_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,1.273159e-07,1,22,1.273159e-07,0.000251,0.987497,0.000011,NaN
1,experiment,3.343757e-05,1,22,3.343757e-05,0.253255,0.619794,0.011381,1.0
2,Interaction,2.605676e-05,1,22,2.605676e-05,0.197353,0.661205,0.008891,NaN


# Supine TMG Analysis

In [28]:
relevant_measurements = ['supine_1_pre', 'supine_2_pre', 'supine_3_pre', 'supine_4_pre',
                         'supine_1_post', 'supine_2_post', 'supine_3_post', 'supine_4_post']
filtered_data = tmg_parameter[tmg_parameter['measurement'].isin(relevant_measurements)]

filtered_data = filtered_data.sort_values(by=['id', 'group', 'experiment']).reset_index(drop=True)
pretest_data = filtered_data[filtered_data['experiment'] == 'pretest']
posttest_data = filtered_data[filtered_data['experiment'] == 'posttest']


In [29]:
pretest_data = pretest_data.reset_index(drop=True)
pretest_data

,id,group,experiment,measurement,Dm,Td,Tc,Ts,Tr,Vc
0,15073,beta_ala,pretest,supine_1_pre,9.726150,31.0,26.0,106.0,72.0,0.110524
1,15073,beta_ala,pretest,supine_2_pre,5.774841,28.0,28.0,98.0,56.0,0.068748
2,15073,beta_ala,pretest,supine_3_pre,9.100387,30.0,27.0,149.0,110.0,0.104602
3,15073,beta_ala,pretest,supine_4_pre,9.112594,29.0,30.0,104.0,64.0,0.103552
4,20438,beta_ala,pretest,supine_1_pre,10.232495,26.0,24.0,44.0,19.0,0.134638
...,...,...,...,...,...,...,...,...,...,...
91,93302,placebo,pretest,supine_4_pre,7.584031,27.0,29.0,51.0,21.0,0.091374
92,93672,beta_ala,pretest,supine_1_pre,11.228515,26.0,27.0,172.0,140.0,0.142133
93,93672,beta_ala,pretest,supine_2_pre,8.410425,25.0,26.0,137.0,103.0,0.110663
94,93672,beta_ala,pretest,supine_3_pre,11.591811,25.0,29.0,144.0,111.0,0.146732


In [30]:
posttest_data = posttest_data.reset_index(drop=True)
posttest_data

,id,group,experiment,measurement,Dm,Td,Tc,Ts,Tr,Vc
0,15073,beta_ala,posttest,supine_1_post,9.949456,31.0,25.0,77.0,46.0,0.114362
1,15073,beta_ala,posttest,supine_2_post,6.965459,28.0,29.0,86.0,46.0,0.081947
2,15073,beta_ala,posttest,supine_3_post,8.982938,30.0,27.0,140.0,104.0,0.103252
3,15073,beta_ala,posttest,supine_4_post,9.926545,30.0,26.0,124.0,88.0,0.115425
4,20438,beta_ala,posttest,supine_1_post,8.599606,27.0,25.0,49.0,21.0,0.108856
...,...,...,...,...,...,...,...,...,...,...
91,93302,placebo,posttest,supine_4_post,8.152583,25.0,28.0,49.0,19.0,0.104520
92,93672,beta_ala,posttest,supine_1_post,10.276126,25.0,24.0,173.0,142.0,0.138867
93,93672,beta_ala,posttest,supine_2_post,9.089581,25.0,25.0,153.0,123.0,0.121194
94,93672,beta_ala,posttest,supine_3_post,11.476495,26.0,28.0,136.0,105.0,0.143456


In [31]:
result_data = pd.DataFrame()

# Hinzufügen von 'group', 'measurement' und 'id'
result_data['group'] = pretest_data['group']
result_data['measurement'] = pretest_data['measurement']
result_data['id'] = pretest_data['id']

# Berechnung der Differenzen für alle Parameter
diff_columns = ['Dm', 'Td', 'Tc', 'Ts', 'Tr', 'Vc']
for col in diff_columns:
    result_data[f'diff_{col}'] = posttest_data[col] - pretest_data[col]

# Anzeigen des Ergebnisses
result_data

,group,measurement,id,diff_Dm,diff_Td,diff_Tc,diff_Ts,diff_Tr,diff_Vc
0,beta_ala,supine_1_pre,15073,0.223306,0.0,-1.0,-29.0,-26.0,0.003837
1,beta_ala,supine_2_pre,15073,1.190617,0.0,1.0,-12.0,-10.0,0.013198
2,beta_ala,supine_3_pre,15073,-0.117449,0.0,0.0,-9.0,-6.0,-0.001350
3,beta_ala,supine_4_pre,15073,0.813951,1.0,-4.0,20.0,24.0,0.011873
4,beta_ala,supine_1_pre,20438,-1.632890,1.0,1.0,5.0,2.0,-0.025782
...,...,...,...,...,...,...,...,...,...
91,placebo,supine_4_pre,93302,0.568552,-2.0,-1.0,-2.0,-2.0,0.013146
92,beta_ala,supine_1_pre,93672,-0.952389,-1.0,-3.0,1.0,2.0,-0.003267
93,beta_ala,supine_2_pre,93672,0.679156,0.0,-1.0,16.0,20.0,0.010531
94,beta_ala,supine_3_pre,93672,-0.115316,1.0,-1.0,-8.0,-6.0,-0.003276


In [32]:
mean_std_supine = result_data.groupby(['group', 'measurement']).agg({
    'diff_Dm': ['mean', 'std'],
    'diff_Td': ['mean', 'std'],
    'diff_Tc': ['mean', 'std'],
    'diff_Ts': ['mean', 'std'],
    'diff_Tr': ['mean', 'std'],
    'diff_Vc': ['mean', 'std'],    
})

mean_std_supine = mean_std_supine.reset_index()
mean_std_supine.columns = ['group', 'measurement',
                           'diff_Dm_mean', 'diff_Dm_std',
                           'diff_Td_mean', 'diff_Td_std',
                           'diff_Tc_mean', 'diff_Tc_std',
                           'diff_Ts_mean', 'diff_Ts_std',
                           'diff_Tr_mean', 'diff_Tr_std',
                           'diff_Vc_mean', 'diff_Vc_std']
mean_std_supine

C:\Users\bvenn\AppData\Local\Temp\ipykernel_11328\1810155329.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_std_supine = result_data.groupby(['group', 'measurement']).agg({


,group,measurement,diff_Dm_mean,diff_Dm_std,diff_Td_mean,diff_Td_std,diff_Tc_mean,diff_Tc_std,diff_Ts_mean,diff_Ts_std,diff_Tr_mean,diff_Tr_std,diff_Vc_mean,diff_Vc_std
0,beta_ala,supine_1_pre,-0.412466,1.231345,-0.083333,0.996205,-1.333333,2.994945,0.583333,25.903521,2.666667,29.370156,-0.002972,0.016618
1,beta_ala,supine_2_pre,0.282381,0.695844,0.333333,2.964436,0.833333,2.691175,5.750000,15.903830,3.750000,16.153384,0.002103,0.013790
2,beta_ala,supine_3_pre,-0.603422,1.364757,0.000000,2.044949,-1.166667,2.855086,6.083333,26.005099,4.916667,28.398010,-0.005608,0.019043
3,beta_ala,supine_4_pre,0.031945,0.772378,0.250000,0.866025,-1.666667,1.874874,-1.583333,48.299178,-1.166667,45.934406,0.001509,0.009516
4,placebo,supine_1_pre,-0.198761,1.967166,0.000000,1.809068,-1.583333,4.699291,-13.750000,41.209277,-9.333333,40.614224,0.000280,0.028788
5,placebo,supine_2_pre,0.355708,1.478547,-0.583333,1.443376,-0.833333,3.809876,-2.583333,23.157399,-1.416667,22.837204,0.006553,0.019645
6,placebo,supine_3_pre,-0.245030,1.698245,0.000000,1.595448,0.333333,2.146173,-19.500000,45.364183,-12.250000,49.900128,-0.002420,0.020470
7,placebo,supine_4_pre,-0.070708,1.736899,-0.333333,1.723281,0.666667,3.700942,-17.000000,45.889195,-15.750000,47.342322,-0.000465,0.023840


In [80]:
# extrahieren des data frames mit differenzen
#result_data.to_excel(r"C:\Users\bvenn\OneDrive\Desktop\Publikation\Dataframe\differences_supine.xlsx")

In [33]:
(
    ggplot(result_data, aes(x='measurement', y='diff_Td', color='group'))
    + geom_boxplot(width=0.5)
    + ylab('Td Difference [ms]')
    + scale_x_discrete(labels=['Supine 1', 'Supine 2', 'Supine 3', 'Supine 4',])
)

In [34]:
# distribution
(
    ggplot(result_data, aes(x='diff_Td'))
    + geom_histogram()
    + ylab('Td Difference [ms]')
)

In [35]:
Td_stat = pg.mixed_anova(dv='diff_Td',between='group', within='measurement', subject='id', data=result_data, correction='auto', effsize='np2')
Td_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,3.010417,1,22,3.010417,0.621992,0.438725,0.027495,NaN
1,measurement,0.197917,3,66,0.065972,0.024890,0.994639,0.001130,0.916476
2,Interaction,4.114583,3,66,1.371528,0.517447,0.671719,0.022980,NaN


In [36]:
(
    ggplot(result_data, aes(x='measurement', y='diff_Tc', color='group'))
    + geom_boxplot(width=0.5)
    + ylab('Tc Difference [ms]')
    + scale_x_discrete(labels=['Supine 1', 'Supine 2', 'Supine 3', 'Supine 4',])
)

In [37]:
# distribution
(
    ggplot(result_data, aes(x='diff_Tc'))
    + geom_histogram()
    + ylab('Td Diff [ms]')
)

In [38]:
Tc_stat = pg.mixed_anova(dv='diff_Tc',between='group', within='measurement', subject='id', data=result_data, correction='auto', effsize='np2')
Tc_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,5.510417,1,22,5.510417,0.355099,0.557323,0.015884,NaN
1,measurement,27.364583,3,66,9.121528,1.057685,0.373139,0.045871,0.876647
2,Interaction,57.697917,3,66,19.232639,2.230116,0.092866,0.092039,NaN


In [39]:
(
    ggplot(result_data, aes(x='measurement', y='diff_Dm', color='group'))
    + geom_boxplot(width=0.5)
    + ylab('Dm Difference [mm]')
    + scale_x_discrete(labels=['Supine 1', 'Supine 2', 'Supine 3', 'Supine 4',])
)

In [40]:
# distribution
(
    ggplot(result_data, aes(x='diff_Dm'))
    + geom_histogram()
    + ylab('Td Diff [ms]')
)

In [41]:
Dm_stat = pg.mixed_anova(dv='diff_Dm',between='group', within='measurement', subject='id', data=result_data, correction='auto', effsize='np2')
Dm_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,0.441899,1,22,0.441899,0.096011,0.759586,0.004345,NaN
1,measurement,7.902471,3,66,2.634157,2.191772,0.097258,0.090600,0.945033
2,Interaction,0.698274,3,66,0.232758,0.193668,0.900355,0.008726,NaN


In [42]:
(
    ggplot(result_data, aes(x='measurement', y='diff_Ts', color='group'))
    + geom_boxplot(width=0.5)
    + ylab('Ts Difference [ms]')
    + scale_x_discrete(labels=['Supine 1', 'Supine 2', 'Supine 3', 'Supine 4',])
)

In [43]:
# distribution
(
    ggplot(result_data, aes(x='diff_Ts'))
    + geom_histogram()
    + ylab('Ts Diff [ms]')
)

In [44]:
Ts_stat = pg.mixed_anova(dv='diff_Ts',between='group', within='measurement', subject='id', data=result_data, correction='auto', effsize='np2')
Ts_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,6080.166667,1,22,6080.166667,3.037713,0.095313,0.121325,NaN
1,measurement,1606.416667,3,66,535.472222,0.507711,0.678321,0.022557,0.82267
2,Interaction,922.250000,3,66,307.416667,0.291479,0.831402,0.013076,NaN


In [45]:
(
    ggplot(result_data, aes(x='measurement', y='diff_Tr', color='group'))
    + geom_boxplot(width=0.5)
    + ylab('Tr Difference [ms]')
    + scale_x_discrete(labels=['Supine 1', 'Supine 2', 'Supine 3', 'Supine 4',])
)

In [46]:
# distribution
(
    ggplot(result_data, aes(x='diff_Tr'))
    + geom_histogram()
    + ylab('Tr Diff [ms]')
)

In [47]:
Tr_stat = pg.mixed_anova(dv='diff_Tr',between='group', within='measurement', subject='id', data=result_data, correction='auto', effsize='np2')
Tr_stat

,Source,SS,DF1,DF2,MS,F,p-unc,p-GG-corr,np2,eps,sphericity,W-spher,p-spher
0,group,3589.260417,1,22,3589.260417,1.681529,0.208158,NaN,0.071006,NaN,NaN,NaN,NaN
1,measurement,1113.531250,3,66,371.177083,0.333728,0.800977,0.748283,0.014943,0.805833,False,0.574002,0.034171
2,Interaction,479.114583,3,66,159.704861,0.143592,0.933427,NaN,0.006485,NaN,NaN,NaN,NaN


In [48]:
(
    ggplot(result_data, aes(x='measurement', y='diff_Vc', color='group'))
    + geom_boxplot(width=0.5)
    + ylab('Vc Difference [mm/ms]')
    + scale_x_discrete(labels=['Supine 1', 'Supine 2', 'Supine 3', 'Supine 4',])
)

In [49]:
# distribution
(
    ggplot(result_data, aes(x='diff_Vc'))
    + geom_histogram()
    + ylab('Vc Diff [mm/ms]')
)

In [50]:
Vc_stat = pg.mixed_anova(dv='diff_Vc',between='group', within='measurement', subject='id', data=result_data, correction='auto', effsize='np2')
Vc_stat

C:\Users\bvenn\AppData\Roaming\Python\Python311\site-packages\pingouin\distribution.py:1004: RuntimeWarning: divide by zero encountered in scalar divide
  W = np.prod(eig) / (eig.sum() / d) ** d


,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,0.000119,1,22,0.000119,0.128531,0.723377,0.005808,NaN
1,measurement,0.000885,3,66,0.000295,1.397414,0.251395,0.059725,0.931854
2,Interaction,0.000147,3,66,0.000049,0.232779,0.873229,0.010470,NaN
